In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-community

In [30]:
%pip install --upgrade --quiet  langchain langchain-google-vertexai "langchain-google-community[featurestore]"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.147.0 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from google.cloud import aiplatform
import google.cloud.bigquery as bq
import langchain
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.schema import format_document
from langchain_community.document_loaders import DirectoryLoader, TextLoader,BigQueryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore

import uuid

In [13]:
# Define our query
query = """
SELECT id,media_type,content,test_metadata 
FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
"""

# Load the data
loader = BigQueryLoader(
    query, metadata_columns=["id"], page_content_columns=["content","media_type","test_metadata"]
)
data = loader.load()

In [8]:
docs = []
docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=20, chunk_overlap  = 5)
docs = text_splitter.split_documents(docs)

In [9]:
doc_ids = [str(uuid.uuid4()) for _ in docs]
chunked_docs=[]
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    doc.metadata['chunk_id']=_id

chunked_docs.extend(docs)

In [36]:
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"

In [12]:
DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}


embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)


store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


In [ ]:
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

### source

https://cloud.google.com/blog/products/ai-machine-learning/rag-with-bigquery-and-langchain-in-cloud

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []
documents.extend(loader.load())
 
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5#,
    #separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
)
doc_splits = text_splitter.split_documents(documents)

# Add chunk number to metadata
chunk_idx=0
prev=doc_splits[0].metadata["id"]
for idx, split in enumerate(doc_splits):
    if prev==split.metadata["id"]:
       split.metadata["chunk"] = chunk_idx
      
    else:
        chunk_idx=0
        split.metadata["chunk"] = chunk_idx
        prev=split.metadata["id"]
    chunk_idx +=1
        

In [37]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore

DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}



embedding_model = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)
bq_store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
) 

bq_store.add_documents(doc_splits)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


['9410869125d64745a5f43c13b3b2b321',
 '090c42e9f05c4e64a97a88efd2aaf917',
 'fbc54c12542748ce978dd32c1ada0ac5',
 '980cb20b73d34bcb9f3b4c7accc6d86f',
 'c58352f83d234699935fe0a6978231f3',
 '62f5e6ab93c649678cc543cb9812200c',
 'a677b2c456694bffa1831332dbce8824',
 'f2e850648fbe4216b60e4c30d1b2750a',
 '513964d9d82246fbb7de0f2252bc5ef7',
 'af42d8a5c5fc49d8861b314e1748794e',
 'c21d5d15bc52410e8a589a46c0aa78a1',
 '21c0f8561eca47e2ac24b7d6a64d90a3',
 '4f4cdd7062854511b75426757cdc1e12',
 '9243289c6cb74a03ad909d87bce094c3',
 '22b2cce24a254a4db3003447e026f826',
 '63cd41511041401b934a4542b52d69da',
 '64ff53f22fec45bb8c9a793bf6e37005',
 '8c0e0f91a61940689b01d8eacd5cb454',
 '2ffcc3f92704462eb346c3579547e451',
 'e83c7136c6b64bc68f65eb841ba503b3',
 '9aa4da4a26964e01a6ce4267a82184df',
 'e1aa10cc289c4d3baaba84e97dc320c4',
 '6d2d0ba19d484b62a69744fee028c654',
 'c9281f5513cc4ce1b0130d1779d5aa89',
 '7275baef07c2423084efae408cba534d',
 'cf41e20dc9e74e008b7406c63c986bc0',
 '9b82500018a945f9af5f7e95464ade6b',
 